<a href="https://colab.research.google.com/github/mpjuarez/practica1_SRC/blob/main/practica2_sis_recomendacion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# > **1. Instalar e importar librerias**



In [187]:
!pip install pandas numpy scikit-surprise
!pip install scipy

**Importación de las librerias**

In [188]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from scipy.sparse import csr_matrix #Para crear la matriz usuario-pelicula
from sklearn.metrics import mean_squared_error
from math import sqrt
import scipy


# 2. **Cargar el conjunto de datos y Presentar características**

In [189]:
#se clona el repositorio de github
!rm -rf practica1_SRC
!git clone https://github.com/mpjuarez/practica1_SRC.git


Cloning into 'practica1_SRC'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 48 (delta 18), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 1.08 MiB | 4.36 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [190]:
#Cargar el dataset de calificaciones, películas y presentar la cantidad de columnas y filas, así tambien los tipos de datos
calificaciones = pd.read_csv('practica1_SRC/ratings.csv')
peliculas = pd.read_csv('practica1_SRC/movies.csv')
print(peliculas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
None


In [191]:
print(calificaciones.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None


In [192]:
# Muestra las primeras 10 filas de calificaciones
print("Primeras 10 filas de calificaciones:")
display(calificaciones.head(10))

# Muestra las primeras 10 filas de películas
print("\nPrimeras 10 filas de películas:")
display(peliculas.head(10))

Primeras 10 filas de calificaciones:


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100



Primeras 10 filas de películas:


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


# **4** Cargar matriz usuario - película

In [193]:
#Matriz Usuario-Película: esto permite reorganizar las calificaciones en una estructura que conecta usuarios con películas.

#!pip install scipy
#from scipy.sparse import csr_matrix

# Crear la matriz usuario-película y llenar NaN con ceros
user_movie_matrix = calificaciones.pivot(index='userId', columns='movieId', values='rating').fillna(0)
print(user_movie_matrix)

# Convertir la matriz a dispersa para mejorar la eficiencia, es decir ahorrar memoria y mejorar la velocidad
user_movie_sparse_matrix = csr_matrix(user_movie_matrix)




movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     0.0     4.0     0.0     0.0     4.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...         ...     ...     ...     ...     ...     ...     ...     ...   
606         2.5     0.0     0.0     0.0     0.0     0.0     2.5     0.0   
607         4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
608         2.5     2.0     2.0     0.0     0.0     0.0     0.0     0.0   
609         3.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
610         5.0     0.0     0.0     0.0     0.0     5.0     0.0     0.0   

movieId  9       10     

# **5. Calcular la matriz de similitud de Pearson**

In [201]:
# Función para calcular la similitud de Pearson
def calcular_similitud_pearson(matrix):
    """
    Calcula la matriz de similitud de Pearson para una matriz de usuario-película.
    """
    # Convertir a denso si es necesario
    matrix_dense = matrix.toarray() if hasattr(matrix, "toarray") else matrix

    # Calcular la media de las calificaciones para cada usuario (ignorando ceros)
    user_means = np.sum(matrix_dense, axis=1) / np.maximum(np.count_nonzero(matrix_dense, axis=1), 1)
    user_means = user_means.reshape(-1, 1)  # Ajustar la forma de user_means

    # Restar la media de cada usuario para centrar las calificaciones
    matrix_centered = np.where(matrix_dense != 0, matrix_dense - user_means, 0)

    # Calcular similitud de Pearson
    numerator = np.dot(matrix_centered, matrix_centered.T)
    denominator = np.sqrt(np.sum(matrix_centered ** 2, axis=1)).reshape(-1, 1)
    denominator = np.dot(denominator, denominator.T)

    # Evitar divisiones por cero
    similarity = np.where(denominator > 0, numerator / denominator, 0)
    return similarity





In [202]:
print("Índices de la matriz usuario-película:", user_movie_matrix.index)
print("Tipo de datos:", type(user_movie_matrix))
print("Usuario solicitado:", user_id)

Índices de la matriz usuario-película: Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       601, 602, 603, 604, 605, 606, 607, 608, 609, 610],
      dtype='int64', name='userId', length=610)
Tipo de datos: <class 'pandas.core.frame.DataFrame'>
Usuario solicitado: 5


In [203]:
print("Índices de la matriz usuario-película:", user_movie_matrix.index)
print("Tipo de datos:", type(user_movie_matrix))
print("Usuario solicitado:", user_id)

Índices de la matriz usuario-película: Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       601, 602, 603, 604, 605, 606, 607, 608, 609, 610],
      dtype='int64', name='userId', length=610)
Tipo de datos: <class 'pandas.core.frame.DataFrame'>
Usuario solicitado: 5


# **6. Función para generar recomendaciones**

In [215]:
# Función para generar predicciones
def generar_predicciones(user_id, matrix):
    #Genera predicciones de calificaciones para todas las películas del usuario.

    #Calcular la matriz de similitud de Pearson
    similarity_matrix = calcular_similitud_pearson(matrix)



    # Identificar los vecinos más cercanos
    user_index = user_id - 1  # Ajustar índice (IDs comienzan desde 1)
    user_similarities = similarity_matrix[user_index]
    neighbors = np.argsort(-user_similarities)[1:10]  # variar número de vecinos

    # Convertir la matriz a densa si es necesario
    matrix_dense = matrix.toarray() if hasattr(matrix, "toarray") else matrix

    # Calcular predicciones
    predicciones = np.zeros(matrix_dense.shape[1])
    user_ratings = matrix_dense[user_index]

    for pelicula in range(matrix_dense.shape[1]):
        score, suma_similitud = 0, 0
        for vecino in neighbors:
            vecino_similitud = user_similarities[vecino]
            vecino_rating = matrix_dense[vecino, pelicula]
            if vecino_rating > 0:
                score += vecino_similitud * vecino_rating
                suma_similitud += vecino_similitud

        if suma_similitud > 0:
            predicciones[pelicula] = score / suma_similitud

    return predicciones


calcular RMSE

In [212]:
# Función para calcular el RMSE
def calcular_rmse(predicciones, calificaciones_reales):
    """
    Calcula el RMSE entre las calificaciones predichas y reales.
    """
    # Filtrar solo las películas calificadas por el usuario
    indices = calificaciones_reales.nonzero()  # Índices de valores no nulos
    reales = calificaciones_reales[indices]
    predichas = predicciones[indices]

    # Calcular RMSE
    mse = mean_squared_error(reales, predichas)
    rmse = sqrt(mse)
    return rmse

Integración del  flujo

# 8. Generar recomendaciones para un usuario

In [213]:
# --- Flujo principal ---

# Supongamos que ya tienes el DataFrame user_movie_matrix
# Convertir DataFrame a matriz dispersa
user_movie_sparse_matrix = csr_matrix(user_movie_matrix.values)

# Generar predicciones para el usuario
user_id = 20  # Usuario para evaluar
predicciones_usuario = generar_predicciones(user_id, user_movie_sparse_matrix)

# Extraer las calificaciones reales del usuario
calificaciones_reales_usuario = user_movie_sparse_matrix[user_id - 1].toarray().flatten()

# Calcular RMSE
rmse = calcular_rmse(predicciones_usuario, calificaciones_reales_usuario)
print(f"RMSE para el usuario {user_id}: {rmse:.2f}")

RMSE para el usuario 20: 1.51


<ipython-input-201-e4ec5d793370>:22: RuntimeWarning: invalid value encountered in divide
  similarity = np.where(denominator > 0, numerator / denominator, 0)


In [214]:
usuarios = range(1, 6)  # Evaluar los primeros 10 usuarios
rmse_total = 0

for user_id in usuarios:
    predicciones_usuario = generar_predicciones(user_id, user_movie_sparse_matrix)
    calificaciones_reales_usuario = user_movie_sparse_matrix[user_id - 1].toarray().flatten()
    rmse = calcular_rmse(predicciones_usuario, calificaciones_reales_usuario)
    print(f"RMSE para el usuario {user_id}: {rmse:.2f}")
    rmse_total += rmse

rmse_promedio = rmse_total / len(usuarios)
print(f"RMSE promedio: {rmse_promedio:.2f}")

<ipython-input-201-e4ec5d793370>:22: RuntimeWarning: invalid value encountered in divide
  similarity = np.where(denominator > 0, numerator / denominator, 0)


RMSE para el usuario 1: 1.43


<ipython-input-201-e4ec5d793370>:22: RuntimeWarning: invalid value encountered in divide
  similarity = np.where(denominator > 0, numerator / denominator, 0)


RMSE para el usuario 2: 2.48


<ipython-input-201-e4ec5d793370>:22: RuntimeWarning: invalid value encountered in divide
  similarity = np.where(denominator > 0, numerator / denominator, 0)


RMSE para el usuario 3: 2.69


<ipython-input-201-e4ec5d793370>:22: RuntimeWarning: invalid value encountered in divide
  similarity = np.where(denominator > 0, numerator / denominator, 0)


RMSE para el usuario 4: 2.38
RMSE para el usuario 5: 1.70
RMSE promedio: 2.14


<ipython-input-201-e4ec5d793370>:22: RuntimeWarning: invalid value encountered in divide
  similarity = np.where(denominator > 0, numerator / denominator, 0)


dasd